# Finetuning LLM using Unsloth

Let's try a simple finetuning phi-3.5-mini-instruct on thinking dataset using unsloth

In [1]:
'''The Unsloth library is a specialized toolkit designed to
significantly accelerate and optimize the fine-tuning of large language models (LLMs). By providing performance enhancements that drastically reduce training time and memory usage, Unsloth makes advanced LLM fine-tuning more accessible to developers using standard or consumer-grade hardware, such as Google Colab GPUs. 
Core functions of the Unsloth library

    Faster and more efficient fine-tuning: Unsloth employs advanced techniques like optimized custom GPU kernels, manual backpropagation, and low-rank adaptation (LoRA) to dramatically improve the speed of fine-tuning. The library claims to make fine-tuning up to 5 times faster with 70% less memory usage compared to traditional methods using the Hugging Face ecosystem.
    Reduced memory consumption: By leveraging 4-bit and 16-bit quantization and other optimizations, Unsloth allows developers to train large models on GPUs with limited VRAM. This makes it possible to fine-tune 7B parameter models on as little as 5GB of VRAM.
    Simplified workflow: Unsloth offers a streamlined, developer-friendly API that simplifies the complex process of fine-tuning. It provides a single class, FastLanguageModel, to handle model loading, quantization, and preparation for PEFT (Parameter-Efficient Fine-Tuning).
    Broad model support: It is compatible with major LLM architectures like Llama, Mistral, Gemma, Phi, and Qwen, and seamlessly integrates with the Hugging Face ecosystem, including its Trainer and SFTTrainer classes.
    Accuracy preservation: Crucially, Unsloth achieves its speed and efficiency gains without sacrificing the model's accuracy. It avoids approximation methods and uses exact computation to ensure the quality of the fine-tuned model.
    Dynamic quantization: The library uses a dynamic quantization method that intelligently selects the best quantization level for each layer of a model. This results in better performance and accuracy during GGUF exports compared to a one-size-fits-all approach.
    Multi-task support: In addition to standard fine-tuning, Unsloth supports other training types, including text-to-speech (TTS), speech-to-text (STT), reinforcement learning (RL), and full fine-tuning.
    Easy deployment: Models fine-tuned with Unsloth can be exported to formats like GGUF, making them easy to deploy on various platforms, including local machines running llama.cpp or on inference engines like vLLM. '''

"The Unsloth library is a specialized toolkit designed to\nsignificantly accelerate and optimize the fine-tuning of large language models (LLMs). By providing performance enhancements that drastically reduce training time and memory usage, Unsloth makes advanced LLM fine-tuning more accessible to developers using standard or consumer-grade hardware, such as Google Colab GPUs. \nCore functions of the Unsloth library\n\n    Faster and more efficient fine-tuning: Unsloth employs advanced techniques like optimized custom GPU kernels, manual backpropagation, and low-rank adaptation (LoRA) to dramatically improve the speed of fine-tuning. The library claims to make fine-tuning up to 5 times faster with 70% less memory usage compared to traditional methods using the Hugging Face ecosystem.\n    Reduced memory consumption: By leveraging 4-bit and 16-bit quantization and other optimizations, Unsloth allows developers to train large models on GPUs with limited VRAM. This makes it possible to fin

In [2]:
!nvidia-smi

Wed Aug 27 16:08:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.76.07              Driver Version: 581.08         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...    On  |   00000000:06:00.0  On |                  N/A |
|  0%   35C    P8              6W /  215W |     618MiB /   8192MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Setup

In [3]:
%%capture
# get the latest nightly Unsloth!
#!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [4]:
!pip install accelerate -U

In [5]:
# !pip uninstall -y packaging
# !pip install packaging==24.1 --force-reinstall
# !pip uninstall unsloth triton -y
# !pip install --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"



In [6]:
from unsloth import FastLanguageModel
import torch
#max_seq_length = 4096
max_seq_length = 512
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/golongson/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3.5-mini-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.8.9: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    NVIDIA GeForce RTX 2070 SUPER. Num GPUs = 1. Max memory: 8.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Data preparation

In [9]:
from datasets import load_dataset

ds = load_dataset("SkunkworksAI/reasoning-0.01",split = "train")
torch.cuda.empty_cache()

In [10]:
ds[0]

{'instruction': 'If a die is rolled three times, what is the probability of getting a sum of 11? None',
 'reasoning': '1. Understand the problem: We need to find the probability of getting a sum of 11 when rolling a die three times.\n2. Calculate total possible outcomes: A die has 6 faces, so for each roll, there are 6 possibilities. For three rolls, the total possible outcomes are 6^3 = 216.\n3. Identify favorable outcomes: List all combinations of rolls that result in a sum of 11. There are 18 such combinations.\n4. Calculate probability: Divide the number of favorable outcomes by the total possible outcomes: 18 / 216 = 1/12.\n5. Conclusion: The probability of getting a sum of 11 when rolling a die three times is 1/12.',
 'output': "To solve this problem, we need to find the number of favorable outcomes (getting a sum of 11) and divide it by the total possible outcomes when rolling a die three times.\n\nFirst, let's find the total possible outcomes. Since a die has six faces, there a

In [11]:
from unsloth.chat_templates import get_chat_template

def formatting_prompts(example):
    reasoning = ""
    t = [{
        "role":"user",
        "content":f"{example['instruction']}"},
        {
        "role":"assistant",
        "content":f"<thinking>{example['reasoning_chains'][0:-1]}</thinking> {example['reasoning_chains'][-1]['thought']}"
    }]
    return t

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "user", "content" : "content", "user" : "user", "assistant" : "assistant"}
)

def formatting_prompts_func(example):
    conversations = formatting_prompts(example)
    texts = tokenizer.apply_chat_template(conversations, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }

In [12]:
dataset = ds.map(formatting_prompts_func, batched = False,)

In [13]:
dataset['text'][0]
torch.cuda.empty_cache()

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 4,
    packing = True, #False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        save_strategy = "steps",
        save_steps=30,
        per_device_train_batch_size = 2, #32,
        gradient_accumulation_steps = 16, #2,
        warmup_steps = 10,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 12,
        output_dir = "outputs_lora_r128_lalpha128",
    ),
)

/home/golongson/miniconda3/envs/llm/lib/python3.11/site-packages/transformers/training_args.py:2083: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/mnt/c/models/llm/5FineTuning/unsloth_compiled_cache/UnslothSFTTrainer.py:481: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/mnt/c/models/llm/5FineTuning/unsloth_compiled_cache/UnslothSFTTrainer.py:569: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/mnt/c/models/llm/5FineTuning/unsloth_compiled_cache/UnslothSFTTrainer.py:583: UserWarning: You passed a `dataset_num_proc` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/mnt/c/models/llm/5FineTuning/unsloth_compiled_cache/UnslothSFTTrai

In [17]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 31,457 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 239,075,328 of 4,060,154,880 (5.89% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.472000
2,1.419300
3,1.389800
4,1.262500
5,1.163700
6,1.034700
7,1.070900
8,1.000400
9,1.027500
10,0.943100


## Inference

In [18]:
torch.cuda.empty_cache()
from unsloth.chat_templates import get_chat_template


def formatting_prompts(example):
    reasoning = ""
    t = [{
        "role":"user",
        "content":f"{example['instruction']}"},
        {
        "role":"assistant",
        "content":f"<thinking>{example['reasoning_chains'][0:-1]}</thinking> {example['reasoning_chains'][-1]['thought']}"
    }]
    return t

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3",
    mapping = {"role" : "user", "content" : "content", "user" : "user", "assistant" : "assistant"}
)

def formatting_prompts_func(example):
    conversations = formatting_prompts(example)
    texts = tokenizer.apply_chat_template(conversations, tokenize = False, add_generation_prompt = False)
    return { "text" : texts, }


In [19]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit

In [20]:
messages = [
    {"role": "user", "content": " If five cats can catch five mice in five minutes, how long will it take one cat to catch one mouse?"},
]

In [21]:
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

In [22]:
outputs = model.generate(input_ids = inputs, max_new_tokens = 4089, use_cache = True)
response = tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [23]:
import re
import json 
import ast

answer = response[0].split("</thinking>")[1]
result = re.search(r'<thinking>(.*?)</thinking>', response[0])
if result:
    t = result.group(1)
    
    outs = t.split(",")
    for d in outs:
        print(d)
print(answer)

[{'step': 1
 'thought': 'The problem states that five cats can catch five mice in five minutes.'}
 {'step': 2
 'thought': 'This implies that each cat is catching one mouse in five minutes.'}
 {'step': 3
 'thought': 'Therefore
 if we have one cat
 it should also be able to catch one mouse in five minutes.'}]
 So, the answer to the problem is that it will take one cat five minutes to catch one mouse.<|end|><|endoftext|>


## Text Streamer 

In [24]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference


messages = [
    {"role": "user", "content": "There is a barrel with no lid and some wine in it. “This barrel of wine is more than half full,” says the woman. “No, it's not,” says the man. “It’s less than half full.” Without any measuring implements and without removing any wine from the barrel, how can they easily determine who is correct?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024, use_cache = True)

<thinking>[{'step': 1, 'thought': 'The problem is about determining whether the wine in the barrel is more than half full or less than half full without any measuring instruments or removing any wine.'}, {'step': 2, 'thought': 'The woman claims the wine is more than half full, while the man claims it is less than half full.'}, {'step': 3, 'thought': 'The only way to determine the truth without any measuring instruments or removing any wine is to use the barrel's lid.'}, {'step': 4, 'thought': 'If the wine is more than half full, the lid will sink into the wine, indicating that the wine is indeed more than half full.'}, {'step': 5, 'thought': 'If the wine is less than half full, the lid will float on the surface of the wine, indicating that the wine is indeed less than half full.'}, {'step': 6, 'thought': 'Therefore, by observing whether the lid sinks or floats, they can easily determine who is correct.'}]</thinking> This solution is based on the principle that the density of a liquid i

#### As you can see from the outputs, the model has adopted the 'thinking' structure in multiple steps when providing the final answer.